# The Leaky Integrate-and-fire

<img src="./lif_formulation.png" alt="drawing" width="400"/>

In [1]:
import numpy as np
from typing import TypeAlias
import time

from lif_hparams import (
    _dt,
    _t_max,
    _tau_m,
    _V_reset,
    _V_thresh,
    _R,
    num_simulations
)

with open('weights.npy', 'rb') as f:
    W = np.load(f)


# Initial conditions
n_neurons = len(W)# Number of neurons in the network
_V = np.ones(n_neurons) * _V_reset  # Initial potentials

from tqdm.notebook import tqdm

# Type Definitions for Clarity

In [2]:
Tensor3D: TypeAlias = np.ndarray
Mat: TypeAlias = np.ndarray
Vec: TypeAlias = np.ndarray 
CurrentArray: TypeAlias = np.ndarray

# Run the Simulations

In [3]:
def run_simulation(
    W: Mat,
    V: Vec,

    # Neuron Parameters
    tau_m: float,
    v_reset: float,
    v_thresh: float,
    membr_R: float,

    # How long do we run for? 
    t_max: float,
    dt: float, 

):
    # Simulation

    spike_train = []
    for i, t in enumerate(np.arange(0, t_max, dt)):
        if i == 0:
            continue
    
        fired = V >= v_thresh
        V = np.where(fired, v_reset, V)
        
        # Record spike times
        spike_train.append(fired)
    
        # Update voltages
        I_syn = W.dot(spike_train[-1])  # Synaptic current from spikes
        dV = (dt / tau_m) * (-V + v_reset + membr_R * I_syn)
        V += dV
    
        # No self-inputs; neurons cannot spike themselves in this timestep
        V = np.where(fired, v_reset, V)
    return spike_train

time_arr = []
for i in range(num_simulations):
    start = time.time()
    spike_train = run_simulation(
        W,
        _V,
        _tau_m, _V_reset, _V_thresh, _R,
        _t_max, _dt
    )
    end = time.time()
    #print(f"Iteration {i} took: {end - start} seconds")
    time_arr.append(end - start)

print(f"Average Time: {np.mean(time_arr)}")
print(f"S.Dev Time: {np.std(time_arr)}")

Average Time: 9.863247315088907
S.Dev Time: 0.570362263795038
